In [1]:
import pandas as pd
import numpy as np
import time
import os
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 5)

In [2]:
df_order = pd.read_csv('export_order.csv')
df_store_fact = pd.read_csv('export_store_fact.csv')
df_order_item = pd.read_csv('export_order_item.csv')

In [3]:
df_store_phase = df_store_fact[(df_store_fact['store_type'] == 'RETAIL_STORE') & (df_store_fact['store_phase'] == 'LIVE') & (df_store_fact['active_flag'] == 1)]
df_store_fact1 = df_store_phase[['store_id','city_name','store_name']]
df_store_fact1

,store_id,city_name,store_name
0,1703e30b-0199-4850-b59a-e0b2eb5396a9,Chennai,CHN-NAVALUR
1,3587d930-d517-4847-8e2c-c48ecabeba45,Chennai,CHN-PALLAVARAM
...,...,...,...
561,6de9aaa0-268d-44a5-8e39-aa2130d5354a,Kolkata,KOL-Kamalgachi
562,4d3f28cf-646c-49cb-833b-0dbce82498d5,Kolkata,KOL-Bhawanipore New


In [4]:
df_order1 = df_order[['id','created_on','store_id','status','type']]
df_order1.rename(columns = {'store_id':'order_store_id'},inplace = True)
df_order1

,id,created_on,order_store_id,status,type
0,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03T15:57:06.495Z,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER
1,dd27767f-5d10-486e-966a-cc41d64d190c,2024-07-03T16:20:12.065Z,c620a3ce-ee2e-493f-a83d-5555531fd861,SHIPPED,CUSTOMER
...,...,...,...,...,...
579689,565c0852-859a-4fbf-a0b5-74975055877c,2024-07-03T08:50:16.608Z,460836cf-9dbb-49e3-b460-ef7fecdb7b13,SHIPPED,CUSTOMER
579690,33f43d02-91a9-4ea4-be74-6470af28d6f4,2024-07-03T08:20:41.036Z,52d8a0f5-1c05-4afc-8259-c3d2d056b565,SHIPPED,CUSTOMER


In [5]:
df_order_item1 = df_order_item[['order_id','product_variant_id','quantity_allocated','quantity_scanned']]
df_order_item1

,order_id,product_variant_id,quantity_allocated,quantity_scanned
0,e0c80fce-eebc-4ab4-9ece-87d8dcda3673,4a0682e7-f6dc-4875-a16a-af12b2b747be,1.0,1.0
1,5371d6db-170e-4590-bb1d-28ff838d6b31,a66ba5b4-313d-4469-becd-01b69bd19fc7,1.0,1.0
...,...,...,...,...
2550542,b40ae7c5-4cd1-42c5-8a12-2c4e12cff21c,f3f10571-ba3e-4610-94d5-3d464cd1171c,1.0,1.0
2550543,39bfec5b-2f1a-420a-9987-b18f70b4a9fb,4db7d95b-1bc9-429a-9456-f4393526796e,12.0,0.0


In [6]:
merge1 = pd.merge(df_order1,df_order_item1, left_on = 'id',right_on = 'order_id', how = 'inner')

In [7]:
merge1 = merge1[merge1['type'] == 'CUSTOMER']
merge1

,id,created_on,order_store_id,status,type,order_id,product_variant_id,quantity_allocated,quantity_scanned
0,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03T15:57:06.495Z,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,328580c9-d0aa-4b0a-af6f-b7e43b28246f,1.0,1.0
1,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03T15:57:06.495Z,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,a08a5a42-1fbc-42ea-b6ac-957a3f5f8bd6,2.0,2.0
...,...,...,...,...,...,...,...,...,...
2550536,33f43d02-91a9-4ea4-be74-6470af28d6f4,2024-07-03T08:20:41.036Z,52d8a0f5-1c05-4afc-8259-c3d2d056b565,SHIPPED,CUSTOMER,33f43d02-91a9-4ea4-be74-6470af28d6f4,b64f96b8-d2e3-49b4-85a7-01da1921b6d4,1.0,NaN
2550537,33f43d02-91a9-4ea4-be74-6470af28d6f4,2024-07-03T08:20:41.036Z,52d8a0f5-1c05-4afc-8259-c3d2d056b565,SHIPPED,CUSTOMER,33f43d02-91a9-4ea4-be74-6470af28d6f4,775a5cd6-d701-4e3e-88d8-602e9f5b5d84,1.0,NaN


In [8]:
df = pd.merge(merge1,df_store_fact1, left_on = 'order_store_id', right_on = 'store_id', how = 'inner')
df

,id,created_on,order_store_id,status,type,order_id,product_variant_id,quantity_allocated,quantity_scanned,store_id,city_name,store_name
0,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03T15:57:06.495Z,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,328580c9-d0aa-4b0a-af6f-b7e43b28246f,1.0,1.0,7c3e8dc3-8781-489c-a2c7-8f8f67149083,Bengaluru,BLR-VIJAY NAGAR
1,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03T15:57:06.495Z,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,a08a5a42-1fbc-42ea-b6ac-957a3f5f8bd6,2.0,2.0,7c3e8dc3-8781-489c-a2c7-8f8f67149083,Bengaluru,BLR-VIJAY NAGAR
...,...,...,...,...,...,...,...,...,...,...,...,...
2077213,c88cb453-d9a7-412b-99d1-c26b192a760b,2024-07-03T08:01:19.138Z,ca648d74-89e2-415d-98d5-7ff5c233898c,SHIPPED,CUSTOMER,c88cb453-d9a7-412b-99d1-c26b192a760b,a4d3642c-ea28-4633-a785-df3f796a4281,1.0,1.0,ca648d74-89e2-415d-98d5-7ff5c233898c,Hyderabad,HYD- Sainikpuri New
2077214,b9893181-cc8c-4e8b-99c6-2e0d7e1b74ba,2024-07-03T08:30:21.063Z,ca648d74-89e2-415d-98d5-7ff5c233898c,SHIPPED,CUSTOMER,b9893181-cc8c-4e8b-99c6-2e0d7e1b74ba,e069e1ad-fd71-46cb-b700-4ec05ff7f76e,1.0,1.0,ca648d74-89e2-415d-98d5-7ff5c233898c,Hyderabad,HYD- Sainikpuri New


In [9]:
df['created_on'] = pd.to_datetime(df['created_on']).dt.strftime('%Y-%m-%d')

In [10]:
df

,id,created_on,order_store_id,status,type,order_id,product_variant_id,quantity_allocated,quantity_scanned,store_id,city_name,store_name
0,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,328580c9-d0aa-4b0a-af6f-b7e43b28246f,1.0,1.0,7c3e8dc3-8781-489c-a2c7-8f8f67149083,Bengaluru,BLR-VIJAY NAGAR
1,b48c94bf-91bb-4d08-ae43-abebdde26a14,2024-07-03,7c3e8dc3-8781-489c-a2c7-8f8f67149083,SHIPPED,CUSTOMER,b48c94bf-91bb-4d08-ae43-abebdde26a14,a08a5a42-1fbc-42ea-b6ac-957a3f5f8bd6,2.0,2.0,7c3e8dc3-8781-489c-a2c7-8f8f67149083,Bengaluru,BLR-VIJAY NAGAR
...,...,...,...,...,...,...,...,...,...,...,...,...
2077213,c88cb453-d9a7-412b-99d1-c26b192a760b,2024-07-03,ca648d74-89e2-415d-98d5-7ff5c233898c,SHIPPED,CUSTOMER,c88cb453-d9a7-412b-99d1-c26b192a760b,a4d3642c-ea28-4633-a785-df3f796a4281,1.0,1.0,ca648d74-89e2-415d-98d5-7ff5c233898c,Hyderabad,HYD- Sainikpuri New
2077214,b9893181-cc8c-4e8b-99c6-2e0d7e1b74ba,2024-07-03,ca648d74-89e2-415d-98d5-7ff5c233898c,SHIPPED,CUSTOMER,b9893181-cc8c-4e8b-99c6-2e0d7e1b74ba,e069e1ad-fd71-46cb-b700-4ec05ff7f76e,1.0,1.0,ca648d74-89e2-415d-98d5-7ff5c233898c,Hyderabad,HYD- Sainikpuri New


In [11]:
final_df = df.groupby(['created_on','city_name','store_name','store_id'])[['quantity_allocated','quantity_scanned']].sum().reset_index()

In [12]:
final_df

,created_on,city_name,store_name,store_id,quantity_allocated,quantity_scanned
0,2024-07-03,Bengaluru,BANASHANKARI,0a357d97-30cb-4eb4-a4b8-0e1fba2471d8,8018.0,7815.0
1,2024-07-03,Bengaluru,BLR - Bhadrappa Layout,6047c8fe-15ed-4c61-8d3e-060e800702f5,5323.0,5289.0
...,...,...,...,...,...,...
355,2024-07-03,Pune,PUN_KOREGAON PARK,596b77f1-86e7-4246-ad37-f87897a9779a,6199.0,6190.0
356,2024-07-03,Pune,Pun-Ravet Network,c00b42da-968d-46af-b09d-8313ed62be85,4807.0,4807.0


In [13]:
final_df.columns

Index(['created_on', 'city_name', 'store_name', 'store_id',
       'quantity_allocated', 'quantity_scanned'],
      dtype='object')

In [14]:
part1 = final_df[['created_on', 'city_name', 'store_name', 'store_id']]
part1['metric_type'] = 'Scan'
part1['metric_name'] = 'Scan_Allocated'
part1['metric_value'] = final_df['quantity_allocated']

part2 = final_df[['created_on', 'city_name', 'store_name', 'store_id']]
part2['metric_type'] = 'Scan'
part2['metric_name'] = 'Scan_Scanned'
part2['metric_value'] = final_df['quantity_scanned']

result = pd.concat([part1, part2], ignore_index=True)
result

,created_on,city_name,store_name,store_id,metric_type,metric_name,metric_value
0,2024-07-03,Bengaluru,BANASHANKARI,0a357d97-30cb-4eb4-a4b8-0e1fba2471d8,Scan,Scan_Allocated,8018.0
1,2024-07-03,Bengaluru,BLR - Bhadrappa Layout,6047c8fe-15ed-4c61-8d3e-060e800702f5,Scan,Scan_Allocated,5323.0
...,...,...,...,...,...,...,...
712,2024-07-03,Pune,PUN_KOREGAON PARK,596b77f1-86e7-4246-ad37-f87897a9779a,Scan,Scan_Scanned,6190.0
713,2024-07-03,Pune,Pun-Ravet Network,c00b42da-968d-46af-b09d-8313ed62be85,Scan,Scan_Scanned,4807.0
